In [1]:
from itertools import combinations
from binance.helpers import round_step_size
from binance.client import Client
from binance.enums import *


from datetime import datetime, timedelta, date, time
from itertools import product
from time import strftime
import scipy.stats
import pandas as pd
import numpy as np
import itertools
import warnings
import os.path
import time
import copy
import json
import os
import ta
import ast


import importlib.util
warnings.filterwarnings("ignore")

In [2]:
PATH_CONDITIONS = r'/Users/westhomas/Desktop/ENKI/ENKI_CODE_DEVELOPMENT/_CONDITIONS/'
PATH_VARIABLES  = r'/Users/westhomas/Desktop/ENKI/ENKI_CODE_DEVELOPMENT/_VARIABLES_/'

PATH_CLEAN      = r'/Users/westhomas/Desktop/ENKI/ENKI_CODE_DEVELOPMENT/A__RAW_DATA_CLEAN/'
PATH_TECHNICAL  = r'/Users/westhomas/Desktop/ENKI/ENKI_CODE_DEVELOPMENT/B__RAW_DATA_TECHNICAL/'
PATH_STRATEGIES = r'/Users/westhomas/Desktop/ENKI/ENKI_CODE_DEVELOPMENT/C__RAW_DATA_STRATEGIES/'
PATH_OUTPUT     = r'/Users/westhomas/Desktop/ENKI/ENKI_CODE_DEVELOPMENT/D__RAW_DATA_OUTPUT/'

In [ ]:


###############################################################################################################################################################################################################################################################################################
######################################################################################################################################  STEP 1  ###############################################################################################################################################
###############################################################################################################################################################################################################################################################################################


####   RAW DATA GATHER                          ################################################################################################################################################################################################################################################
####   TECHNICAL INDICATORS ESTABLISHING        ################################################################################################################################################################################################################################################
####   TP AND SL ESTABLISHING                   ################################################################################################################################################################################################################################################








class HISTORICAL_DATA():
    def __init__(self, client, TICKER, TIMEFRAME):
        self.client         = client
        self.TICKER         = TICKER
        self.TIMEFRAME      = TIMEFRAME
        self.DF             = self.get_data()

    def get_data(self):

        INTERVALS               =  {'4H'    : {'FUNCTION': self.client.KLINE_INTERVAL_4HOUR},
                                    '1D'    : {'FUNCTION': self.client.KLINE_INTERVAL_1DAY}}

        END                     = (datetime.now() + timedelta(days = 1)).date()
        START                   = END - timedelta(days = 1750)

        candle                  = (np.asarray(self.client.get_historical_klines(self.TICKER, INTERVALS[self.TIMEFRAME]['FUNCTION'], str(START), str(END))))[:, :6]
        candle                  = pd.DataFrame(candle, columns=['datetime', 'open', 'high', 'low', 'close', 'volume']).astype(float).rename(columns={'datetime':'DATE', 'open':'OPEN', 'high':'HIGH', 'low':'LOW', 'close':'CLOSE', 'volume':'VOLUME'})
        candle.DATE             = pd.to_datetime(candle.DATE, unit='ms')
        return candle




class DATA_PROCESS():
    def __init__(self, DF, UPPER_DF):
        self.DF                                 = DF
        self.UPPER_DF                           = UPPER_DF
        self.DF                                 = self.DATA_PROCESS()

    def DATA_PROCESS(self):
        OHLC_DICT                                                                   = {'OPEN' : 'first',   'HIGH' : 'max',  'LOW' : 'min',  'CLOSE' : 'last',  'VOLUME' : 'sum'}
        self.DF                                                                     = self.DF.set_index('DATE')
        self.DF                                                                     = self.DF.resample(self.UPPER_DF).agg(OHLC_DICT)
        COLS                                                                        = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME']
        for q in range(len(COLS)):              self.DF[COLS[q]]                    = pd.to_numeric(self.DF[COLS[q]])
        self.DF                                                                     = self.DF.reset_index()
        return self.DF
    







class TECHNICAL_ANALYSIS:
    def __init__(self, DF):
        self.DF = DF
        self.DF = self.TECHNICAL_ANALYSIS()

    def TECHNICAL_ANALYSIS(self):
        TA_PARAMS_WIDER_SCOPE                       = { 'EMA_NBRS': [4, 8, 12, 18, 24, 36, 50, 100, 200],
                                                        'RSI': [14, 20, 30]}
        STD_WINDOW                                  = [5, 10, 20]

        for k in TA_PARAMS_WIDER_SCOPE['EMA_NBRS']: self.DF[f'EMA_{k}'] = self.DF['CLOSE'].ewm(span=k, adjust=False).mean()
        for k in TA_PARAMS_WIDER_SCOPE['RSI']:      
            self.DF[f'RSI_{k}']                     = ta.momentum.RSIIndicator(self.DF['CLOSE'], window=k).rsi()
            self.DF[f'RSI_{k}_DIVERGENCE']          = self.DF['CLOSE'].pct_change() - self.DF[f'RSI_{k}'].pct_change()

        macd                                        = ta.trend.MACD(self.DF['CLOSE'])
        self.DF['MACD']                             = macd.macd()
        self.DF['MACD_SIGNAL']                      = macd.macd_signal()
        self.DF['MACD_DIFF']                        = macd.macd_diff()

        bollinger                                   = ta.volatility.BollingerBands(self.DF['CLOSE'], window=20, window_dev=2)
        self.DF['BB_UPPER']                         = bollinger.bollinger_hband()
        self.DF['BB_MIDDLE']                        = bollinger.bollinger_mavg()
        self.DF['BB_LOWER']                         = bollinger.bollinger_lband()
        self.DF['BB_WIDTH']                         = self.DF['BB_UPPER'] - self.DF['BB_LOWER']

        self.DF['VWAP']                             = (self.DF['VOLUME'] * (self.DF['HIGH'] + self.DF['LOW'] + self.DF['CLOSE']) / 3).cumsum() / self.DF['VOLUME'].cumsum()
        self.DF['CCI']                              = ta.trend.CCIIndicator(high=self.DF['HIGH'], low=self.DF['LOW'], close=self.DF['CLOSE'], window=20).cci()
        self.DF['WILLIAMS_%R']                      = ta.momentum.WilliamsRIndicator(high=self.DF['HIGH'], low=self.DF['LOW'], close=self.DF['CLOSE'], lbp=14).williams_r()
        adx                                         = ta.trend.ADXIndicator(high=self.DF['HIGH'], low=self.DF['LOW'], close=self.DF['CLOSE'], window=14)
        self.DF['ADX']                              = adx.adx()
        self.DF['ADX_NEG']                          = adx.adx_neg()
        self.DF['ADX_POS']                          = adx.adx_pos()
        self.DF['PSAR']                             = ta.trend.PSARIndicator(high=self.DF['HIGH'], low=self.DF['LOW'], close=self.DF['CLOSE']).psar()
        self.DF['TSI']                              = ta.momentum.TSIIndicator(self.DF['CLOSE']).tsi()
        self.DF['MFI']                              = ta.volume.MFIIndicator(high=self.DF['HIGH'], low=self.DF['LOW'], close=self.DF['CLOSE'], volume=self.DF['VOLUME'], window=14).money_flow_index()
        self.DF['ROC']                              = ta.momentum.ROCIndicator(self.DF['CLOSE'], window=12).roc()
        STOCH                                       = ta.momentum.StochasticOscillator(self.DF['HIGH'], self.DF['LOW'], self.DF['CLOSE'])
        self.DF['%K']                               = (STOCH.stoch())
        self.DF['%D']                               = (STOCH.stoch_signal())
        self.DF['VOLATILITY']                       = (((self.DF['HIGH'] - self.DF['LOW']) / self.DF['CLOSE']) * 100)

        self.DF['HAMMER']                           = ((self.DF['CLOSE'] > self.DF['OPEN']) & ((self.DF['LOW'] - self.DF['OPEN']) > 2 * (self.DF['CLOSE'] - self.DF['OPEN']))).astype(int)
        self.DF['HEAD_SHOULDERS']                   = ((self.DF['HIGH'].shift(1) > self.DF['HIGH'].shift(2)) & (self.DF['HIGH'] < self.DF['HIGH'].shift(1)) & (self.DF['HIGH'].shift(2) > self.DF['HIGH'].shift(3))).astype(int)
        self.DF['INV_HEAD_SHOULDERS']               = ((self.DF['LOW'].shift(1) < self.DF['LOW'].shift(2)) &  (self.DF['LOW'] > self.DF['LOW'].shift(1)) & (self.DF['LOW'].shift(2) < self.DF['LOW'].shift(3))).astype(int)
        self.DF['DOUBLE_TOP']                       = ((self.DF['HIGH'] == self.DF['HIGH'].shift(1)) & (self.DF['CLOSE'] < self.DF['CLOSE'].shift(1))).astype(int)
        self.DF['DOUBLE_BOTTOM']                    = ((self.DF['LOW'] == self.DF['LOW'].shift(1)) & (self.DF['CLOSE'] > self.DF['CLOSE'].shift(1))).astype(int)
        self.DF['BULL_FLAG']                        = ((self.DF['CLOSE'] > self.DF['OPEN']) & (self.DF['CLOSE'].shift(1) > self.DF['OPEN'].shift(1)) & (self.DF['CLOSE'] > self.DF['CLOSE'].rolling(window=5).mean())).astype(int)
        self.DF['BEAR_FLAG']                        = ((self.DF['CLOSE'] < self.DF['OPEN']) & (self.DF['CLOSE'].shift(1) < self.DF['OPEN'].shift(1)) & (self.DF['CLOSE'] < self.DF['CLOSE'].rolling(window=5).mean())).astype(int)
        self.DF['ASC_TRIANGLE']                     = ((self.DF['CLOSE'] > self.DF['CLOSE'].rolling(window=20).mean()) & (self.DF['LOW'] > self.DF['LOW'].rolling(window=20).min())).astype(int)
        self.DF['DESC_TRIANGLE']                    = ((self.DF['CLOSE'] < self.DF['CLOSE'].rolling(window=20).mean()) & (self.DF['HIGH'] < self.DF['HIGH'].rolling(window=20).max())).astype(int)
        self.DF['SHOOTING_STAR']                    = ((self.DF['CLOSE'] < self.DF['OPEN']) & ((self.DF['HIGH'] - self.DF['CLOSE']) > 2 * (self.DF['OPEN'] - self.DF['CLOSE']))).astype(int)
        self.DF['DOJI']                             = ((abs(self.DF['CLOSE'] - self.DF['OPEN']) / (self.DF['HIGH'] - self.DF['LOW'])) < 0.1).astype(int)
        self.DF['BULL_ENGULF']                      = ((self.DF['CLOSE'] > self.DF['OPEN']) & (self.DF['CLOSE'].shift(1) < self.DF['OPEN'].shift(1)) & (self.DF['OPEN'] < self.DF['CLOSE'].shift(1))).astype(int)
        self.DF['BEAR_ENGULF']                      = ((self.DF['CLOSE'] < self.DF['OPEN']) & (self.DF['CLOSE'].shift(1) > self.DF['OPEN'].shift(1)) & (self.DF['OPEN'] > self.DF['CLOSE'].shift(1))).astype(int)
        self.DF['VOL_DIVERGENCE']                   = ((self.DF['CLOSE'].diff() * self.DF['VOLUME'].diff()) < 0).astype(int)

        PERIODS                                     = [7, 10, 20]
        COLS_TO_MOVE_BACK                           = ['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME']
        FIB_RATIOS                                  = {"FIB_100" : 1.0, "FIB_1618" : 1.618, "FIB_2618" : 2.618, "FIB_4236" : 4.236}

        for P in PERIODS:
            self.DF[f'RVOL_{P}']                    = self.DF['VOLUME'] / self.DF['VOLUME'].rolling(window=P).mean()
            self.DF[f'PRICE_QUANTILE_{P}']          = self.DF['CLOSE'].rolling(window=P).apply(lambda x: pd.Series(x).rank(pct=True).iloc[-1])
            self.DF[f'VOLUME_SPIKE_{P}']            = (self.DF['VOLUME'] > 2 * self.DF['VOLUME'].rolling(window=P).mean()).astype(int)
            self.DF[f'PRICE_Z_SCORE_{P}']           = (self.DF['CLOSE'] - self.DF['CLOSE'].rolling(window=P).mean()) / self.DF['CLOSE'].rolling(window=P).std()

        for LABEL, RATIO in FIB_RATIOS.items():     self.DF[LABEL] = ((self.DF["HIGH"] - self.DF["LOW"]) * RATIO)
        for a in range(len(STD_WINDOW)):            self.DF[f'STD_{STD_WINDOW[a]}'] = self.DF['CLOSE'].rolling(STD_WINDOW[a]).std()
        for COL in COLS_TO_MOVE_BACK:               self.DF[COL]  = self.DF[COL].shift(-1)

        self.DF                                     = self.DF.iloc[200:, :].reset_index(drop=True)
        return self.DF





class APPLY_TP_SL():
    def __init__(self, DF, TP_SL_OPTIONS, REWARD_FACTOR, RL_RW_OPTIONS, STD_WINDOW):
        self.DF                                     = DF
        self.TP_SL_OPTIONS                          = TP_SL_OPTIONS
        self.REWARD_FACTOR                          = REWARD_FACTOR
        self.RL_RW_OPTIONS                          = RL_RW_OPTIONS
        self.STD_WINDOW                             = STD_WINDOW
        self.DF                                     = self.APPLY_TP_SL()

    def APPLY_TP_SL(self):


        FIB_RATIOS = {"FIB_100": 1.0, "FIB_1618": 1.618, "FIB_2618": 2.618, "FIB_4236": 4.236}


        for idx, (TP_OP, SL_OP) in enumerate(self.TP_SL_OPTIONS, start=1):
            self.DF[f'TP_LONG_{idx}']    = self.DF['OPEN'] * (1 + (TP_OP/100))
            self.DF[f'SL_LONG_{idx}']    = self.DF['OPEN'] * (1 - (SL_OP/100))
            self.DF[f'TP_SHORT_{idx}']   = self.DF['OPEN'] * (1 - (TP_OP/100))
            self.DF[f'SL_SHORT_{idx}']   = self.DF['OPEN'] * (1 + (SL_OP/100))


        for idx, (RL, RF) in enumerate(self.RL_RW_OPTIONS, start=1):
            self.DF[f'TP_LONG_RL_{idx}']                = self.DF['OPEN'] + (((RL/100) * self.DF['OPEN']) * RF)
            self.DF[f'SL_LONG_RL_{idx}']                = self.DF['OPEN'] - ((RL/100) * self.DF['OPEN'])
            self.DF[f'TP_SHORT_RL_{idx}']               = self.DF['OPEN'] - (((RL/100) * self.DF['OPEN']) * RF)
            self.DF[f'SL_SHORT_RL_{idx}']               = self.DF['OPEN'] + ((RL/100) * self.DF['OPEN'])

            for a in range(len(self.STD_WINDOW)):
                self.DF[f'TP_LONG_RL_STD_{idx}_{a}']    = self.DF['OPEN'] + (RF * self.DF[f'STD_{self.STD_WINDOW[a]}'])
                self.DF[f'SL_LONG_RL_STD_{idx}_{a}']    = self.DF['OPEN'] - (RL * self.DF[f'STD_{self.STD_WINDOW[a]}'])
                self.DF[f'TP_SHORT_RL_STD_{idx}_{a}']   = self.DF['OPEN'] - (RF * self.DF[f'STD_{self.STD_WINDOW[a]}'])
                self.DF[f'SL_SHORT_RL_STD_{idx}_{a}']   = self.DF['OPEN'] + (RL * self.DF[f'STD_{self.STD_WINDOW[a]}'])

        for FIB in FIB_RATIOS:
            self.DF[f"TP_LONG_{FIB}"]                   = self.DF["LOW"].shift(1) + self.DF[FIB]
            self.DF[f"SL_LONG_{FIB}"]                   = self.DF["LOW"].shift(1)
            self.DF[f"TP_SHORT_{FIB}"]                  = self.DF["HIGH"].shift(1) - self.DF[FIB]
            self.DF[f"SL_SHORT_{FIB}"]                  = self.DF["HIGH"].shift(1)

        return self.DF





class DEVELOPMENT_CONTROL():

    def __init__(self, COINS, TIMEFRAME, CLIENT):
        self.COINS                              = COINS
        self.TIMEFRAME                          = TIMEFRAME
        self.CLIENT                             = CLIENT
        self.LIST                               = self.DEVELOPMENT_CONTROL()


    def DEVELOPMENT_CONTROL(self):
        COIN_HIERARCHY  = []
        EXPORT          = 'YES'

        STD_WINDOW                                  = [5, 10, 20]
        RISK_LEVELS                                 = [0.5, 1, 1.5, 2]
        REWARD_FACTOR                               = [0.5, 1, 1.5, 2]
        TP                                          = [1, 1.5, 2, 2.5, 5, 10]
        SL                                          = [1, 1.5, 2, 2.5, 5, 10]
        TP_SL_OPTIONS                               = list(itertools.product(TP, SL))
        RL_RW_OPTIONS                               = list(itertools.product(RISK_LEVELS, REWARD_FACTOR))

        VARIABLES                                   = [STD_WINDOW, TP_SL_OPTIONS, RL_RW_OPTIONS]
        VARIABLES_STR                               = ['STD_WINDOW', 'TP_SL_OPTIONS', 'RL_RW_OPTIONS']


        for e in range(len(VARIABLES)):
            FILEPATH = PATH_CONDITIONS + VARIABLES_STR[e] + '.txt'
            with open(FILEPATH, "w") as file: file.write(str(VARIABLES[e]))



        for COIN_USE in self.COINS:
            TIME_FRAME_HIERARCHY                = []
            RAW                                 = HISTORICAL_DATA(self.CLIENT, COIN_USE, '4H')

            for TIMEFRAME_USE in self.TIMEFRAME:
                FILENAME                        = f'{COIN_USE} -- {TIMEFRAME_USE}.csv'
                MULTI_RAW                       = DATA_PROCESS(RAW.DF, TIMEFRAME_USE)
                if EXPORT == 'YES':             MULTI_RAW.DF.to_csv((PATH_CLEAN + FILENAME), index=False)


                TA_DF                           = TECHNICAL_ANALYSIS(MULTI_RAW.DF)
                TP_SL_DF                        = APPLY_TP_SL(TA_DF.DF, TP_SL_OPTIONS, REWARD_FACTOR, RL_RW_OPTIONS, STD_WINDOW)
                if EXPORT == 'YES':             TP_SL_DF.DF.to_csv((PATH_TECHNICAL + FILENAME), index=False)


                TIME_FRAME_HIERARCHY.append(TP_SL_DF.DF)
            COIN_HIERARCHY.append(TIME_FRAME_HIERARCHY)

        return COIN_HIERARCHY







###############################################################################################################################################################################################################################################################################################
####################################################################################################################################  API KEYS  ###############################################################################################################################################
###############################################################################################################################################################################################################################################################################################
MASTER_API_KEY  = 'PFL78Ps4WnSGGhZRcDQ3AFkzBPko9jA2B3d7lasLgDBI8VREV0QdHPYWaVuSzint'
MASTER_API_SEC  = 'wO9zNzEK7LmgJ8bfcnkh60miN785lWaA4S34XydNMEzq18e90d5fTK92vbdIRYg2'
CLIENT          = Client(MASTER_API_KEY, MASTER_API_SEC)


###############################################################################################################################################################################################################################################################################################
#################################################################################################################################### PARAMETERS ###############################################################################################################################################
###############################################################################################################################################################################################################################################################################################

TIMEFRAME       = ['4H', '1D']
COIN            = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT', 'ADAUSDT', 'AVAXUSDT', 'LTCUSDT', 'RLCUSDT']

###############################################################################################################################################################################################################################################################################################
####################################################################################################################################   COMMAND  ###############################################################################################################################################
###############################################################################################################################################################################################################################################################################################

NEEDED = True
if NEEDED == True: DATA = DEVELOPMENT_CONTROL(COIN, TIMEFRAME, CLIENT)


In [3]:
class READ_IN_DATA():

    def __init__(self, COINS, TIMEFRAME, PATH):
        self.COINS                              = COINS
        self.TIMEFRAME                          = TIMEFRAME
        self.PATH                               = PATH
        self.LIST                               = self.READ_IN_RAW_DATA()


    def READ_IN_RAW_DATA(self):
        
        COIN_HIERARCHY = []
        for COIN_USE in self.COINS:

            TIME_FRAME_HIERARCHY = []
            for TIMEFRAME_USE in self.TIMEFRAME:

                FILENAME                    = f'{COIN_USE} -- {TIMEFRAME_USE}.csv'
                DF                          = (pd.read_csv((self.PATH + FILENAME))).iloc[:-1]
                DF["DATE"]                  = pd.to_datetime(DF["DATE"])


                TIME_FRAME_HIERARCHY.append(DF)
            COIN_HIERARCHY.append(TIME_FRAME_HIERARCHY)

        return COIN_HIERARCHY







class EXPORT_RAW_DATA():

    def __init__(self, DF, FILEPATH):
        self.DF                     = DF
        self.FILEPATH               = FILEPATH
        self.DF                     = self.READ_IN_RAW_DATA()

    def READ_IN_RAW_DATA(self):

        self.DF.to_csv(self.FILEPATH, index=False)
        print(f"Data has been exported to - {self.FILEPATH}")

        return self.DF









def LIST_COMBINATIONS_GROUPING(LIST_OF_LISTS, PATH, FILENAME, MAX_LENGTH):


    for i, USEFUL_STRATEGIES_LIST in enumerate(LIST_OF_LISTS):

        TEXT_FILE_PATH                  = PATH + f'{FILENAME}{i}.txt'
        BATCH_SIZE, COMBINATION_BUFFER  = 50000, []
        ALL_ELEMENTS                    = list(itertools.chain(*USEFUL_STRATEGIES_LIST.values()))
        ELEMENTS_TO_CATEGORY            = {ELEM: KEY for KEY, ELEMENTS in USEFUL_STRATEGIES_LIST.items() for ELEM in ELEMENTS}


        with open(TEXT_FILE_PATH, 'w') as FILE:
            for R in range(1, min(MAX_LENGTH, len(ALL_ELEMENTS) + 1)):
                for COMBO in itertools.combinations(ALL_ELEMENTS, R):

                    CAT_IN_COMBO = {ELEMENTS_TO_CATEGORY[ELEM] for ELEM in COMBO}

                    if len(CAT_IN_COMBO) == len(COMBO): COMBINATION_BUFFER.append(f"{list(COMBO)}\n")
                    if len(COMBINATION_BUFFER) >= BATCH_SIZE:
                        FILE.writelines(COMBINATION_BUFFER)
                        COMBINATION_BUFFER = [] 

            if COMBINATION_BUFFER: FILE.writelines(COMBINATION_BUFFER)
            
    return COMBINATION_BUFFER







def READ_IN_FILES(PATH, LIST_OF_LISTS_0):

    LIST_STRATEGIES_COMBO   = []

    for i, USEFUL_STRATEGIES_LIST in enumerate(LIST_OF_LISTS_0): 
        with open((PATH + f'VARIABLES_{i}.txt'), "r") as FILE:   LIST_STRATEGIES_COMBO.append([ast.literal_eval(LINE.strip()) for LINE in FILE])

    FLATTENED_STRATEGIES        = list(itertools.chain(*LIST_STRATEGIES_COMBO))
    UNIQUE_STRATEGIES, SEEN     = [], set() 

    for STRATEGY in FLATTENED_STRATEGIES:
        STRATEGY_TUPLE = tuple(STRATEGY)  
        if STRATEGY_TUPLE not in SEEN:
            SEEN.add(STRATEGY_TUPLE)
            UNIQUE_STRATEGIES.append(STRATEGY)


    return UNIQUE_STRATEGIES













In [12]:


def STRATEGY_GROUPINGS(PATH, DATA_DF):

    STRATEGY_COLUMNS                = [COL for COL in DATA_DF.columns if COL.startswith("STRATEGY_") 
                                        and not COL.startswith(('STRATEGY_EMA_', 'STRATEGY_SMA_', 'STRATEGY_PREV_GAIN_DOWN', 'STRATEGY_PREV_GAIN_UP', 'STRATEGY_PRICE_QUANTILE_', 'STRATEGY_Z_SCORE_ANOMALY_', 
                                                                'STRATEGY_PRICE_Z_SCORE_', 'STRATEGY_RVOL_', 'STRATEGY_ROC_', 'STRATEGY_CCI_', 'STRATEGY_WILLIAMS_', 'STRATEGY_MFI_', 'STRATEGY_MACD_', 'STRATEGY_BB',
                                                                'STRATEGY_STOCHASTIC_', 'STRATEGY_STOCHASTIC_CROSSOVER_', 'STRATEGY_STOCHASTIC_COMBINED_', 'STRATEGY_RSI_', 'STRATEGY_AVG_VOLATILITY_', 'STRATEGY_ENTROPY_', 'STRATEGY_ADX', 'STRATEGY_VWAP'))]


    STRATEGY_EMA_                   = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_EMA_')] 
    STRATEGY_PRICE_QUANTILE_        = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_PRICE_QUANTILE_')] 
    STRATEGY_PRICE_Z_SCORE_         = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_PRICE_Z_SCORE_')] 
    STRATEGY_RVOL_                  = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_RVOL_')] 
    STRATEGY_CCI_                   = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_CCI_')] 
    STRATEGY_WILLIAMS_              = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_WILLIAMS_')] 
    STRATEGY_STOCHASTIC_            = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_STOCHASTIC_')]
    STRATEGY_RSI_                   = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_RSI_') and not COL.endswith(('_DIVERGENCE'))]
    STRATEGY_MACD_                  = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_MACD_')]
    STRATEGY_BB                     = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_BB')]
    STRATEGY_ADX                    = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_ADX')]
    STRATEGY_VWAP                   = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_VWAP')]



    ###### 1. Trend-Following + Momentum ######
    LIST_1  = {'STRATEGY_EMA_'              : STRATEGY_EMA_,           #   Why: EMA confirms the trend, MACD confirms momentum, and RSI avoids overbought/oversold trades.
                'STRATEGY_MACD_'            : STRATEGY_MACD_,
                'STRATEGY_RSI_'             : STRATEGY_RSI_}


    LIST_2  = {'STRATEGY_ADX'               : STRATEGY_ADX,
                'STRATEGY_WILLIAMS_'        : STRATEGY_WILLIAMS_,
                'STRATEGY_STOCHASTIC_'      : STRATEGY_STOCHASTIC_}


    ###### 2. Volatility + Trend ######
    LIST_3  = {'STRATEGY_BB'                : STRATEGY_BB,               #   Why: CCI confirms cyclical movements alongside volatility-based breakout indicators.
                'STRATEGY_CCI_'             : STRATEGY_CCI_,
                'STRATEGY_PRICE_Z_SCORE_'   : STRATEGY_PRICE_Z_SCORE_}

    LIST_4  = {'STRATEGY_EMA_'              : STRATEGY_EMA_,             #   Why: VWAP and EMA detect price levels, and RSI avoids overbought/oversold trades.
                'STRATEGY_VWAP'             : STRATEGY_VWAP,
                'STRATEGY_RSI_'             : STRATEGY_RSI_}

    ###### 3. Momentum + Volatility ######
    LIST_5  = {'STRATEGY_RSI_'              : STRATEGY_RSI_,           #   Why: RSI measures momentum, and RVOL highlights periods of high trading activity, confirming momentum strength.
                'STRATEGY_RVOL_'            : STRATEGY_RVOL_,
                'STRATEGY_PRICE_Z_SCORE_'   : STRATEGY_PRICE_Z_SCORE_}


    LIST_7  = {'STRATEGY_CCI_'              : STRATEGY_CCI_,        #   Why: VWAP and CCI detect price cyclicality and conditions, while RSI avoids weak momentum trades.
                'STRATEGY_VWAP'             : STRATEGY_VWAP,
                'STRATEGY_RSI_'             : STRATEGY_RSI_}


    ###### 4. Overbought/Oversold + Divergence ######
    LIST_8  = {'STRATEGY_RSI_'              : STRATEGY_RSI_,                #   Why: RSI spots overbought/oversold levels, and MACD divergence confirms reversals.
                'STRATEGY_MACD_'            : STRATEGY_MACD_,
                'STRATEGY_BB'               : STRATEGY_BB}


    ###### 5. Pattern Recognition + Volatility ######
    LIST_10 = {'STRATEGY_COLUMNS'           : STRATEGY_COLUMNS,              #   Why: Double bottoms indicate reversals, and RVOL confirms accumulation phases or breakout volumes.
                'STRATEGY_RVOL_'            : STRATEGY_RVOL_,
                'STRATEGY_BB'               : STRATEGY_BB,
                'STRATEGY_PRICE_Z_SCORE_'   : STRATEGY_PRICE_Z_SCORE_,
                'STRATEGY_PRICE_QUANTILE_'  : STRATEGY_PRICE_QUANTILE_}


    FILENAME            = 'VARIABLES_'
    LIST_OF_LISTS       = [LIST_1, LIST_2, LIST_3, LIST_4,LIST_5, LIST_7, LIST_8, LIST_10]
    MAX_LENGTH          = 5 if len(LIST_OF_LISTS) > 2 else 25
    OUTPUT              = LIST_COMBINATIONS_GROUPING(LIST_OF_LISTS, PATH, FILENAME, MAX_LENGTH)
    UNIQUE_STRATEGIES   = READ_IN_FILES(PATH, LIST_OF_LISTS)
    
    return UNIQUE_STRATEGIES




TIMEFRAME               = ['1D']
COIN                    = ['BTCUSDT']
DATA                    = READ_IN_DATA(COIN, TIMEFRAME, PATH_TECHNICAL)
DATA_DF                 = DATA.LIST[0][0].copy()
STRATEGY_DF             = STRATEGY_DEVELOPMENT(DATA_DF)
UNIQUE_STRATEGIES       = STRATEGY_GROUPINGS(PATH_VARIABLES, STRATEGY_DF)
len(UNIQUE_STRATEGIES)





11459

In [15]:

def STRATEGY_DEVELOPMENT(DF):

    LIST        = [4, 8, 12, 18, 24, 36, 50, 100, 200]
    PAIR_COMBO  = list(combinations(LIST, 2))
    ROC_OPTIONS = [0.5, 1, 2, 5]
    RSI_OPTIONS = [10, 20, 30, 40]
    RSI_LIST    = [14, 20, 30]
    PERIODS     = [7, 10, 20]
    CCI_OPTIONS = [1, 2]
    FIELDS      = ['EMA']


    for FIELD in FIELDS:
        for I in range(len(PAIR_COMBO)):
            LOWER                                   = f'{FIELD}_{PAIR_COMBO[I][0] if PAIR_COMBO[I][0] < PAIR_COMBO[I][1] else PAIR_COMBO[I][1]}'
            UPPER                                   = f'{FIELD}_{PAIR_COMBO[I][1] if PAIR_COMBO[I][0] < PAIR_COMBO[I][1] else PAIR_COMBO[I][0]}'
            DF[f'STRATEGY_{LOWER}__{UPPER}']        = np.where((DF[LOWER].shift(1) < DF[UPPER].shift(1)) & (DF[LOWER] > DF[UPPER]), 'LONG', np.where((DF[LOWER].shift(1) > DF[UPPER].shift(1)) & (DF[LOWER] < DF[UPPER]), 'SHORT','STATIC'))


    for I in range(len(RSI_LIST)):
        for G in range(len(RSI_OPTIONS)):
            RSI_LOWER                               = RSI_OPTIONS[G]
            RSI_UPPER                               = 100 - RSI_OPTIONS[G]
            RSI_COLUMN                              = f'RSI_{RSI_LIST[I]}'

            DF[f'STRATEGY_RSI_{RSI_LIST[I]}__{G}']  = np.where(DF[RSI_COLUMN] < RSI_LOWER, 'LONG',np.where(DF[RSI_COLUMN] > RSI_UPPER, 'SHORT', 'STATIC'))


    DF['STRATEGY_MACD_SIGNAL_LINE']                 = np.where( (DF['MACD'] > DF['MACD_SIGNAL']), 'LONG',np.where(DF['MACD'] < DF['MACD_SIGNAL'], 'SHORT', 'STATIC'))
    DF['STRATEGY_MACD_ZERO_LINE_STRATEGY']          = np.where(DF['MACD'] > 0, 'LONG',np.where(DF['MACD'] < 0, 'SHORT', 'STATIC'))
    DF['STRATEGY_MACD_DIFF_STRATEGY']               = np.where(DF['MACD_DIFF'] > 0, 'LONG',np.where(DF['MACD_DIFF'] < 0, 'SHORT', 'STATIC'))
    DF['STRATEGY_MACD_COMBINED_STRATEGY']           = np.where((DF['MACD'] > DF['MACD_SIGNAL']) & (DF['MACD'] > 0) & (DF['MACD_DIFF'] > 0), 'LONG',np.where((DF['MACD'] < DF['MACD_SIGNAL']) & (DF['MACD'] < 0) & (DF['MACD_DIFF'] < 0), 'SHORT','STATIC'))
    DF['STRATEGY_BB_MEAN_REVERSION']                = np.where(DF['CLOSE'].shift(1) <= DF['BB_LOWER'], 'LONG',np.where(DF['CLOSE'].shift(1) >= DF['BB_UPPER'], 'SHORT', 'STATIC'))
    DF['STRATEGY_BB_BREAKOUT']                      = np.where((DF['CLOSE'].shift(1) > DF['BB_UPPER']) & (DF['BB_WIDTH'] > DF['BB_WIDTH'].rolling(20).mean()), 'LONG',np.where((DF['CLOSE'].shift(1) < DF['BB_LOWER']) & (DF['BB_WIDTH'] > DF['BB_WIDTH'].rolling(20).mean()), 'SHORT','STATIC'))
    DF['STRATEGY_BB_MIDDLE_REVERSION']              = np.where((DF['CLOSE'].shift(2) < DF['BB_MIDDLE']) & (DF['CLOSE'].shift(1) > DF['BB_MIDDLE']), 'LONG',np.where((DF['CLOSE'].shift(2) > DF['BB_MIDDLE']) & (DF['CLOSE'].shift(1) < DF['BB_MIDDLE']), 'SHORT','STATIC'))
    DF['STRATEGY_VWAP']                             = np.where(DF['CLOSE'].shift(1) > DF['VWAP'], 'LONG',np.where(DF['CLOSE'].shift(1) < DF['VWAP'], 'SHORT', 'STATIC'))
    DF['STRATEGY_PSAR']                             = np.where(DF['CLOSE'].shift(1) > DF['PSAR'], 'LONG',np.where(DF['CLOSE'].shift(1) < DF['PSAR'], 'SHORT', 'STATIC'))
    DF['STRATEGY_TSI']                              = np.where(DF['TSI'] > 0, 'LONG',np.where(DF['TSI'] < 0, 'SHORT', 'STATIC'))
    DF['STRATEGY_ADX']                              = np.where((DF['ADX_POS'] > DF['ADX_NEG']) & (DF['ADX'] > DF['ADX'].shift(1)), 'LONG',np.where((DF['ADX_NEG'] > DF['ADX_POS']) & (DF['ADX'] > DF['ADX'].shift(1)), 'SHORT','STATIC'))

    for G in range(len(RSI_OPTIONS)):
        LOWER                                       = RSI_OPTIONS[G]
        UPPER                                       = 100 - RSI_OPTIONS[G]
        DF[f'STRATEGY_WILLIAMS_%R_{G}']             = np.where(DF['WILLIAMS_%R'] < (-1*UPPER), 'LONG',np.where(DF['WILLIAMS_%R'] > (-1*LOWER), 'SHORT', 'STATIC'))
        DF[f'STRATEGY_MFI_{G}']                     = np.where(DF['MFI'] < LOWER, 'LONG',np.where(DF['MFI'] > UPPER, 'SHORT', 'STATIC'))
        DF[f'STRATEGY_STOCHASTIC_{G}']              = np.where((DF['%K'] < LOWER) & (DF['%K'] > DF['%K'].shift(1)), 'LONG',  np.where((DF['%K'] > UPPER) & (DF['%K'] < DF['%K'].shift(1)), 'SHORT', 'STATIC'))
        DF[f'STRATEGY_STOCHASTIC_CROSSOVER_{G}']    = np.where((DF['%K'].shift(1) < DF['%D'].shift(1)) & (DF['%K'] > DF['%D']), 'LONG', np.where((DF['%K'].shift(1) > DF['%D'].shift(1)) & (DF['%K'] < DF['%D']), 'SHORT', 'STATIC'))
        DF[f'STRATEGY_STOCHASTIC_COMBINED_{G}']     = np.where(((DF['%K'] < LOWER) & (DF['%K'] > DF['%K'].shift(1))) | ((DF['%K'].shift(1) < DF['%D'].shift(1)) & (DF['%K'] > DF['%D'])), 'LONG',
                                                        np.where(((DF['%K'] > UPPER) & (DF['%K'] < DF['%K'].shift(1))) | ((DF['%K'].shift(1) > DF['%D'].shift(1)) & (DF['%K'] < DF['%D'])), 'SHORT','STATIC'))

    for G in range(len(CCI_OPTIONS)):               DF[f'STRATEGY_CCI_{G}'] = np.where(DF['CCI'] < (-1*(100 / CCI_OPTIONS[G])), 'LONG',np.where(DF['CCI'] > ((100 / CCI_OPTIONS[G])), 'SHORT', 'STATIC'))
    for G in range(len(ROC_OPTIONS)):               DF[f'STRATEGY_ROC_{G}'] = np.where(DF['ROC'] > ROC_OPTIONS[G], 'LONG',np.where(DF['ROC'] < (-1*ROC_OPTIONS[G]), 'SHORT', 'STATIC'))

    DF['STRATEGY_HAMMER']                           = np.where(DF['HAMMER'] == 1, 'LONG', 'STATIC')
    DF['STRATEGY_INV_HEAD_SHOULDERS']               = np.where(DF['INV_HEAD_SHOULDERS'] == 1, 'LONG', 'STATIC')
    DF['STRATEGY_DOUBLE_BOTTOM']                    = np.where(DF['DOUBLE_BOTTOM'] == 1, 'LONG', 'STATIC')
    DF['STRATEGY_BULL_FLAG']                        = np.where(DF['BULL_FLAG'] == 1, 'LONG', 'STATIC')
    DF['STRATEGY_ASC_TRIANGLE']                     = np.where(DF['ASC_TRIANGLE'] == 1, 'LONG', 'STATIC')
    DF['STRATEGY_BULL_ENGULF']                      = np.where(DF['BULL_ENGULF'] == 1, 'LONG', 'STATIC')
    DF['STRATEGY_HEAD_SHOULDERS']                   = np.where(DF['HEAD_SHOULDERS'] == 1, 'SHORT', 'STATIC')
    DF['STRATEGY_DOUBLE_TOP']                       = np.where(DF['DOUBLE_TOP'] == 1, 'SHORT', 'STATIC')
    DF['STRATEGY_BEAR_FLAG']                        = np.where(DF['BEAR_FLAG'] == 1, 'SHORT', 'STATIC')
    DF['STRATEGY_DESC_TRIANGLE']                    = np.where(DF['DESC_TRIANGLE'] == 1, 'SHORT', 'STATIC')
    DF['STRATEGY_SHOOTING_STAR']                    = np.where(DF['SHOOTING_STAR'] == 1, 'SHORT', 'STATIC')
    DF['STRATEGY_BEAR_ENGULF']                      = np.where(DF['BEAR_ENGULF'] == 1, 'SHORT', 'STATIC')

    for G in range(len(PERIODS)):
        CHOSEN = PERIODS[G]
        DF[f'STRATEGY_RVOL_{G}']                         = np.where(DF[f'RVOL_{CHOSEN}'] > 1, 'LONG',np.where(DF[f'RVOL_{CHOSEN}'] < 1, 'SHORT', 'STATIC'))
        DF[f'STRATEGY_PRICE_Z_SCORE_{G}']                = np.where(DF[f'PRICE_Z_SCORE_{CHOSEN}'] < -2, 'LONG',np.where(DF[f'PRICE_Z_SCORE_{CHOSEN}'] > 2, 'SHORT', 'STATIC'))

        for H in range(len(RSI_OPTIONS)):
            LOWER                                       = RSI_OPTIONS[H]/100
            UPPER                                       = 1 - (RSI_OPTIONS[H]/100)
            DF[f'STRATEGY_PRICE_QUANTILE_{G}_{H}']      = np.where(DF[f'PRICE_QUANTILE_{CHOSEN}'] < LOWER, 'LONG',np.where(DF[f'PRICE_QUANTILE_{CHOSEN}'] > UPPER, 'SHORT', 'STATIC'))


    return DF




def MAIN_LOOP_CLEAN(DATA, UNIQUE_STRATEGIES, TIME_USE):

    DATA_DF                                                     = STRATEGY_DEVELOPMENT(DATA)
    STRATEGY_COLUMNS                                            = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_')]
    TP_COLS                                                     = [COL for COL in DATA_DF.columns if COL.startswith('TP_')] 
    SL_COLS                                                     = [COL for COL in DATA_DF.columns if COL.startswith('SL_')] 
    LIST_RETAIN                                                 = ['DATE', "OPEN", "HIGH", "LOW", "CLOSE", "VOLUME"]
    STRATEGY_COLUMNS                                            = LIST_RETAIN + STRATEGY_COLUMNS + TP_COLS + SL_COLS
    DATA_DF                                                     = DATA_DF[STRATEGY_COLUMNS]

    if TIME_USE == '4H':                                        MIN_THRESHOLD = 50
    else:                                                       MIN_THRESHOLD = 25


    for I, COMBO_ELEMENTS in enumerate(UNIQUE_STRATEGIES):      
        DATA_DF[f'STRATEGY_NBR_{I+1}'] = DATA_DF[COMBO_ELEMENTS].apply(lambda row: "LONG" if all(val == "LONG" for val in row) else "SHORT" if all(val == "SHORT" for val in row) else "STATIC",axis=1)

        COUNT_LONG_SHORT = DATA_DF[f'STRATEGY_NBR_{I+1}'].isin(["LONG", "SHORT"]).sum()
        if COUNT_LONG_SHORT < MIN_THRESHOLD: DATA_DF.drop(columns=[f'STRATEGY_NBR_{I+1}'], inplace=True)


    STRATEGY_NBR_                                               = [COL for COL in DATA_DF.columns if COL.startswith('STRATEGY_NBR_')] 
    OUTPUT_COLS                                                 = LIST_RETAIN + STRATEGY_NBR_ + TP_COLS + SL_COLS
    FINAL_REVIEW                                                = DATA_DF[OUTPUT_COLS]

    return FINAL_REVIEW, STRATEGY_NBR_







In [16]:

STRATEGY_LOOP               = True
TIMEFRAME                   = ['1D']
COIN                        = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT', 'ADAUSDT', 'AVAXUSDT', 'LTCUSDT', 'RLCUSDT']
DATA                        = READ_IN_DATA(COIN, TIMEFRAME, PATH_TECHNICAL)


print('LOADED')


if STRATEGY_LOOP == True:
    for a in range(len(COIN)):
        for b in range(len(TIMEFRAME)):


            OUTPUT_DF, STRATEGY_NBR_            = MAIN_LOOP_CLEAN(DATA.LIST[a][b], UNIQUE_STRATEGIES, TIMEFRAME[b])
            FILENAME                            = f'{COIN[a]} -- {TIMEFRAME[b]}.csv'
            DF_OUT                              = EXPORT_RAW_DATA(OUTPUT_DF, (PATH_STRATEGIES + FILENAME))

            print(f'{FILENAME}  --  NBR STRATEGIES : {len(STRATEGY_NBR_)}')








TIMEFRAME                   = ['4H']
COIN                        = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT', 'ADAUSDT', 'AVAXUSDT', 'LTCUSDT', 'RLCUSDT']
DATA                        = READ_IN_DATA(COIN, TIMEFRAME, PATH_TECHNICAL)


print('LOADED')


if STRATEGY_LOOP == True:
    for a in range(len(COIN)):
        for b in range(len(TIMEFRAME)):


            OUTPUT_DF, STRATEGY_NBR_            = MAIN_LOOP_CLEAN(DATA.LIST[a][b], UNIQUE_STRATEGIES, TIMEFRAME[b])
            FILENAME                            = f'{COIN[a]} -- {TIMEFRAME[b]}.csv'
            DF_OUT                              = EXPORT_RAW_DATA(OUTPUT_DF, (PATH_STRATEGIES + FILENAME))

            print(f'{FILENAME}  --  NBR STRATEGIES : {len(STRATEGY_NBR_)}')





LOADED
Data has been exported to - /Users/westhomas/Desktop/ENKI/ENKI_CODE_DEVELOPMENT/C__RAW_DATA_STRATEGIES/BTCUSDT -- 1D.csv
BTCUSDT -- 1D.csv  --  NBR STRATEGIES : 987
Data has been exported to - /Users/westhomas/Desktop/ENKI/ENKI_CODE_DEVELOPMENT/C__RAW_DATA_STRATEGIES/ETHUSDT -- 1D.csv
ETHUSDT -- 1D.csv  --  NBR STRATEGIES : 976
Data has been exported to - /Users/westhomas/Desktop/ENKI/ENKI_CODE_DEVELOPMENT/C__RAW_DATA_STRATEGIES/XRPUSDT -- 1D.csv
XRPUSDT -- 1D.csv  --  NBR STRATEGIES : 1008
Data has been exported to - /Users/westhomas/Desktop/ENKI/ENKI_CODE_DEVELOPMENT/C__RAW_DATA_STRATEGIES/DOGEUSDT -- 1D.csv
DOGEUSDT -- 1D.csv  --  NBR STRATEGIES : 925
Data has been exported to - /Users/westhomas/Desktop/ENKI/ENKI_CODE_DEVELOPMENT/C__RAW_DATA_STRATEGIES/ADAUSDT -- 1D.csv
ADAUSDT -- 1D.csv  --  NBR STRATEGIES : 939
Data has been exported to - /Users/westhomas/Desktop/ENKI/ENKI_CODE_DEVELOPMENT/C__RAW_DATA_STRATEGIES/AVAXUSDT -- 1D.csv
AVAXUSDT -- 1D.csv  --  NBR STRATEGIES : 85

In [19]:
TP_COLS                                                     = [COL for COL in OUTPUT_DF.columns if COL.startswith('TP_')] 


In [50]:


class BACKTEST_NEW():
    def __init__(self, INPUT_LIST, START_DATE, LEVERAGE, PERCENTAGE_USED, STARTING_BALANCE):
        self.INPUT_LIST         = INPUT_LIST
        self.START_DATE         = START_DATE
        self.LEVERAGE           = LEVERAGE
        self.PERCENTAGE_USED    = PERCENTAGE_USED
        self.STARTING_BALANCE   = STARTING_BALANCE
        self.API_DATA           = self.BACKTEST()

    def BACKTEST(self):
        DF_UPPER                    = self.INPUT_LIST[1]
        DF_LOWER                    = self.INPUT_LIST[0]
        self.START_DATE             = pd.to_datetime(self.START_DATE)
        DF_UPPER                    = DF_UPPER[DF_UPPER['DATE'] >= self.START_DATE].reset_index(drop=True)
        DF_UPPER['STR']             = (DF_UPPER['DATE'].dt.date).astype("string") + ' ' + (DF_UPPER['DATE'].dt.time).astype("string")
        DF_LOWER['STR']             = (DF_LOWER['DATE'].dt.date).astype("string") + ' ' + (DF_LOWER['DATE'].dt.time).astype("string")
        DATA_MAIN                   = DF_UPPER[['STR', 'SIGNAL', 'TP', 'SL']]
        FINAL_DF                    = DF_LOWER.merge(DATA_MAIN, how='left', on=['STR'])
        FINAL_DF['SIGNAL']          = FINAL_DF['SIGNAL'].fillna('STATIC')
        FINAL_DF['TRADE_STATUS']    = None 

        TRADES_TP_SL                = self.RUN_BACKTEST(FINAL_DF)
        TRADES_NEXT_TRADE           = self.EXIT_AT_NEXT_TRADE(FINAL_DF)
        TRADES_COMBINED             = self.COMBINED(TRADES_TP_SL, TRADES_NEXT_TRADE)
        SCENARIOS                   = ['TP_SL_ONLY', 'LEADING_ONLY', 'TP_SL_LEADING']
        SCENARIOS_VARIABLES         = [TRADES_TP_SL, TRADES_NEXT_TRADE, TRADES_COMBINED]
        TIME_LIST                   = []

        for A in range(len(SCENARIOS)): 
            if A == 0:              

                SCENARIOS_VARIABLES[A]  = self.REMOVE_OVERLAPPING_TRADES(SCENARIOS_VARIABLES[A])
                PORTFOLIO_RESULTS       = self.PORTFOLIO_TIMELINE(SCENARIOS_VARIABLES[A])
                OVERVIEW                = self.OVERVIEW(SCENARIOS_VARIABLES[A], SCENARIOS[A])
                OVERVIEW_DF             = OVERVIEW.copy()

            else:
            
                PORTFOLIO_RESULTS       = self.PORTFOLIO_TIMELINE(SCENARIOS_VARIABLES[A])
                OVERVIEW                = self.OVERVIEW(SCENARIOS_VARIABLES[A], SCENARIOS[A])             
                OVERVIEW_DF             = pd.concat([OVERVIEW_DF, OVERVIEW], ignore_index=True)


            TIME_LIST.append(PORTFOLIO_RESULTS)

        return OVERVIEW_DF  #[OVERVIEW_DF, TIME_LIST, SCENARIOS_VARIABLES, FINAL_DF]


    def COMBINED(self, TRADES_TP_SL, TRADES_NEXT_TRADE):

        TRADES_COMBINED                         = TRADES_TP_SL.merge(TRADES_NEXT_TRADE, on="START_INDEX", how="inner")
        TRADES_COMBINED['END_INDEX']            = np.where(TRADES_COMBINED['END_INDEX_x'] < TRADES_COMBINED['END_INDEX_y'], TRADES_COMBINED['END_INDEX_x'], TRADES_COMBINED['END_INDEX_y'])
        TRADES_COMBINED['STATUS']               = np.where(TRADES_COMBINED['END_INDEX_x'] < TRADES_COMBINED['END_INDEX_y'], TRADES_COMBINED['STATUS_x'], TRADES_COMBINED['STATUS_y'])   
        TRADES_COMBINED['TIME_COMPLETED']       = np.where(TRADES_COMBINED['END_INDEX_x'] < TRADES_COMBINED['END_INDEX_y'], TRADES_COMBINED['TIME_COMPLETED_x'], TRADES_COMBINED['TIME_COMPLETED_y'])   
        TRADES_COMBINED['PERCENTAGE_CHANGE']    = np.where(TRADES_COMBINED['END_INDEX_x'] < TRADES_COMBINED['END_INDEX_y'], TRADES_COMBINED['PERCENTAGE_CHANGE_x'], TRADES_COMBINED['PERCENTAGE_CHANGE_y'])  
        TRADES_COMBINED                         = TRADES_COMBINED[['START_INDEX', 'END_INDEX', 'STATUS', 'TIME_COMPLETED', 'PERCENTAGE_CHANGE']]

        return TRADES_COMBINED


    def RUN_BACKTEST(self, FINAL_DF):
        RESULTS, ACTIVE_TRADE = [], None

        for IDX, ROW in FINAL_DF.iterrows():
            if ACTIVE_TRADE is None and ROW['SIGNAL'] != "STATIC":
                ACTIVE_TRADE = {"INDEX": IDX, "TRADE": ROW}

            if ACTIVE_TRADE:
                OUTCOME, COMPLETED_TIME, COMPLETED_INDEX, PERCENTAGE_CHANGE = self.TRADE_ANALYSIS(ACTIVE_TRADE["TRADE"], FINAL_DF.iloc[IDX:])

                if OUTCOME in ["SUCCESS", "FAILURE"]:
                    RESULTS.append({"START_INDEX"       : ACTIVE_TRADE["INDEX"],
                                    "END_INDEX"         : COMPLETED_INDEX,
                                    "STATUS"            : OUTCOME,
                                    "TIME_COMPLETED"    : COMPLETED_TIME,
                                    "PERCENTAGE_CHANGE" : PERCENTAGE_CHANGE})

                    FINAL_DF.at[ACTIVE_TRADE["INDEX"], 'TRADE_STATUS'] = OUTCOME
                    ACTIVE_TRADE = None  

        return pd.DataFrame(RESULTS)




    def TRADE_ANALYSIS(self, ROW, TRADE_DATA):
        DIRECTION, TRADE_START, TRADE_OPEN      = ROW['SIGNAL'], ROW['DATE'], ROW['OPEN']
        TRADE_TP, TRADE_SL                      = ROW['TP'], ROW['SL']
        if TRADE_DATA.empty:                    return "FAILURE", TRADE_START, TRADE_DATA.index[-1], 0

        if DIRECTION == "LONG":
            TP_IDX = TRADE_DATA.loc[TRADE_DATA['HIGH'] >= TRADE_TP, 'DATE'].first_valid_index()
            SL_IDX = TRADE_DATA.loc[TRADE_DATA['LOW'] <= TRADE_SL, 'DATE'].first_valid_index()
        else:  
            TP_IDX = TRADE_DATA.loc[TRADE_DATA['LOW'] <= TRADE_TP, 'DATE'].first_valid_index()
            SL_IDX = TRADE_DATA.loc[TRADE_DATA['HIGH'] >= TRADE_SL, 'DATE'].first_valid_index()

        TP_HIT = TRADE_DATA.at[TP_IDX, 'DATE'] if TP_IDX in TRADE_DATA.index else None
        SL_HIT = TRADE_DATA.at[SL_IDX, 'DATE'] if SL_IDX in TRADE_DATA.index else None

        if TP_IDX is not None and SL_IDX is not None:       return ("SUCCESS", TP_HIT, TP_IDX, abs((TRADE_TP - TRADE_OPEN) / TRADE_OPEN) * 100) if TP_IDX < SL_IDX else ("FAILURE", SL_HIT, SL_IDX, -abs((TRADE_SL - TRADE_OPEN) / TRADE_OPEN) * 100)
        elif TP_IDX is not None:                            return "SUCCESS", TP_HIT, TP_IDX, abs((TRADE_TP - TRADE_OPEN) / TRADE_OPEN) * 100
        elif SL_IDX is not None:                            return "FAILURE", SL_HIT, SL_IDX, -abs((TRADE_SL - TRADE_OPEN) / TRADE_OPEN) * 100

        return "FAILURE", TRADE_DATA['DATE'].iloc[-1], TRADE_DATA.index[-1], 0



    def EXIT_AT_NEXT_TRADE(self, FINAL_DF):

        TRADE_DF                = (FINAL_DF[FINAL_DF['SIGNAL'] != "STATIC"])
        TRADE_DF["END_INDEX"]  = TRADE_DF.index.to_series().shift(-1)
        TRADE_DF["EXIT_DATE"]   = TRADE_DF["DATE"].shift(-1)
        TRADE_DF["EXIT_PRICE"]  = TRADE_DF["OPEN"].shift(-1)

        TRADE_DF.iloc[-1, TRADE_DF.columns.get_loc("END_INDEX")]   = FINAL_DF.index[-1]
        TRADE_DF.iloc[-1, TRADE_DF.columns.get_loc("EXIT_DATE")]    = FINAL_DF["DATE"].iloc[-1]
        TRADE_DF.iloc[-1, TRADE_DF.columns.get_loc("EXIT_PRICE")]   = FINAL_DF["CLOSE"].iloc[-1]

        TRADE_DF["STATUS"]              = np.where(((TRADE_DF["EXIT_PRICE"] > TRADE_DF["OPEN"]) & (TRADE_DF["SIGNAL"] == "LONG")) | ((TRADE_DF["EXIT_PRICE"] < TRADE_DF["OPEN"]) & (TRADE_DF["SIGNAL"] == "SHORT")),  "SUCCESS", "FAILURE")
        TRADE_DF["PERCENTAGE_CHANGE"]   = np.where(TRADE_DF["SIGNAL"] == "LONG", ((TRADE_DF["EXIT_PRICE"] - TRADE_DF["OPEN"]) / TRADE_DF["OPEN"]) * 100, ((TRADE_DF["OPEN"] - TRADE_DF["EXIT_PRICE"]) / TRADE_DF["OPEN"]) * 100)

        return TRADE_DF[["END_INDEX", "STATUS", "EXIT_DATE", "PERCENTAGE_CHANGE"]].reset_index().rename(columns={"index": "START_INDEX", "EXIT_DATE": "TIME_COMPLETED"})




    def PORTFOLIO_TIMELINE(self, INPUT_DF):


        BALANCE, N                      = self.STARTING_BALANCE, len(INPUT_DF)
        FEE_RATE, ALLOCATION            = 0.00075, 0.985 if self.PERCENTAGE_USED == 1 else self.PERCENTAGE_USED
        if INPUT_DF.empty:              return pd.DataFrame(columns=["START_AMOUNT", "TRADE_AMOUNT", "LEVERED_AMOUNT", "PERCENTAGE_CHANGE", "GROSS_PROFIT", "FEES", "NET_PROFIT", "END_AMOUNT"])

        start_amounts                   = np.zeros(N)
        trade_amounts                   = np.zeros(N)
        levered_amounts                 = np.zeros(N)
        gross_profits                   = np.zeros(N)
        fees                            = np.zeros(N)
        net_profits                     = np.zeros(N)
        end_amounts                     = np.zeros(N)
        start_amounts[0]                = BALANCE 
        percentage_changes              = INPUT_DF["PERCENTAGE_CHANGE"].values / 100  
        zero_balance_index              = None  

        for i in range(N):
            if zero_balance_index is not None: start_amounts[i] = trade_amounts[i] = levered_amounts[i] = gross_profits[i] = fees[i] = net_profits[i] = end_amounts[i] = 0
            else:
                trade_amounts[i]        = start_amounts[i] * ALLOCATION
                levered_amounts[i]      = trade_amounts[i] * self.LEVERAGE
                gross_profits[i]        = levered_amounts[i] * percentage_changes[i]
                fees[i]                 = levered_amounts[i] * FEE_RATE
                net_profits[i]          = gross_profits[i] - fees[i]
                end_amounts[i]          = start_amounts[i] + net_profits[i]

                if end_amounts[i] <= 0:
                    zero_balance_index  = i 
                    end_amounts[i]      = 0  

            if i < N - 1 and zero_balance_index is None:
                start_amounts[i + 1]    = end_amounts[i]

        INPUT_DF["START_AMOUNT"]        = start_amounts
        INPUT_DF["TRADE_AMOUNT"]        = trade_amounts
        INPUT_DF["LEVERED_AMOUNT"]      = levered_amounts
        INPUT_DF["GROSS_PROFIT"]        = gross_profits
        INPUT_DF["FEES"]                = fees
        INPUT_DF["NET_PROFIT"]          = net_profits
        INPUT_DF["END_AMOUNT"]          = end_amounts

        return INPUT_DF[["START_AMOUNT", "TRADE_AMOUNT", "LEVERED_AMOUNT", "PERCENTAGE_CHANGE", "GROSS_PROFIT", "FEES", "NET_PROFIT", "END_AMOUNT"]]




    def OVERVIEW(self, RESULTS_TABLE, SCENARIO_DESC):

        if RESULTS_TABLE.empty:     return pd.DataFrame([{"METHOD": SCENARIO_DESC, "END_BALANCE": self.STARTING_BALANCE, "WIN_RATE": 0, "NBR_TRADES": 0}])
        WIN_TRADES                  = (RESULTS_TABLE["PERCENTAGE_CHANGE"] > 0).sum()
        TOTAL_TRADES                = len(RESULTS_TABLE)
        WIN_RATE                    = WIN_TRADES / TOTAL_TRADES if TOTAL_TRADES > 0 else 0

        return pd.DataFrame([{  "METHOD"        : SCENARIO_DESC,
                                "END_BALANCE"   : RESULTS_TABLE["END_AMOUNT"].iloc[-1],
                                "WIN_RATE"      : WIN_RATE,
                                "NBR_TRADES"    : TOTAL_TRADES}])
    


    def REMOVE_OVERLAPPING_TRADES(self, DF):

        DF_USE = DF.copy()
        DF_USE = DF_USE.sort_values("START_INDEX").reset_index(drop=True)
        NON_OVERLAPPING = []  
        
        for i in range(len(DF_USE)):
            if i == 0:          NON_OVERLAPPING.append(DF_USE.iloc[i]) 
            else:
                PREV_TRADE      = NON_OVERLAPPING[-1]
                CURRENT_TRADE   = DF_USE.iloc[i]

                if PREV_TRADE["END_INDEX"] > CURRENT_TRADE["START_INDEX"]:     continue  
                else:    NON_OVERLAPPING.append(CURRENT_TRADE)
        
        return pd.DataFrame(NON_OVERLAPPING)





In [20]:
import ast
import re



def DATA_PULL():
    folder_path = r'/Users/westhomas/Desktop/ENKI/ENKI_CODE_DEVELOPMENT/_CONDITIONS/'
    parsed_data = {}


    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):  
            file_path = os.path.join(folder_path, file_name)
            
            with open(file_path, "r") as file:
                content = file.read().strip()  
                try:                            parsed_data[file_name] = ast.literal_eval(content) 
                except Exception as e:          print(f"Error parsing {file_name}: {e}")

    
    TP_SL_OPTIONS   = parsed_data['TP_SL_OPTIONS.txt']
    RL_RW_OPTIONS   = parsed_data['RL_RW_OPTIONS.txt']
    STD_WINDOW      = parsed_data['STD_WINDOW.txt']
    FIB_RATIOS      = {"FIB_100" : 1.0, "FIB_1618" : 1.618, "FIB_2618" : 2.618, "FIB_4236" : 4.236}
    
    return TP_SL_OPTIONS, RL_RW_OPTIONS, STD_WINDOW, FIB_RATIOS




def ANALYSIS(STATUS, TP_SL_OPTIONS, RL_RW_OPTIONS, STD_WINDOW, FIB_RATIOS):

    def categorize_tp_sl(column_name):

        match                           = re.match(r"STATUS__TP_LONG_(\d+)$", column_name)
        if match:
            idx                         = int(match.group(1)) - 1
            return {"Category"          : "Fixed TP/SL",
                    "TP_Value"          : TP_SL_OPTIONS[idx][0] if idx < len(TP_SL_OPTIONS) else None,
                    "SL_Value"          : TP_SL_OPTIONS[idx][1] if idx < len(TP_SL_OPTIONS) else None}

        match                           = re.match(r"STATUS__TP_LONG_RL_(\d+)$", column_name)
        if match:
            idx                         = int(match.group(1)) - 1
            return {"Category"          : "Risk Level TP/SL",
                    "Risk_Level"        : RL_RW_OPTIONS[idx][0] if idx < len(RL_RW_OPTIONS) else None,
                    "Reward_Factor"     : RL_RW_OPTIONS[idx][1] if idx < len(RL_RW_OPTIONS) else None}

        match                           = re.match(r"STATUS__TP_LONG_RL_STD_(\d+)_(\d+)$", column_name)
        if match:
            risk_idx, std_idx           = int(match.group(1)) - 1, int(match.group(2))
            return {"Category"          : "Std Dev TP/SL",
                    "Risk_Level"        : RL_RW_OPTIONS[risk_idx][0] if risk_idx < len(RL_RW_OPTIONS) else None,
                    "Reward_Factor"     : RL_RW_OPTIONS[risk_idx][1] if risk_idx < len(RL_RW_OPTIONS) else None,
                    "Std_Window"        : STD_WINDOW[std_idx] if std_idx < len(STD_WINDOW) else None}

        match                           = re.match(r"STATUS__TP_LONG_FIB_(\d+)$", column_name)
        if match:
            fib_key                     = f"FIB_{match.group(1)}"
            return {"Category"          : "Fibonacci TP/SL",
                    "Fibonacci_Level"   : FIB_RATIOS.get(fib_key, None)}

        return {"Category"              : "Unknown",
                "Description"           : "No matching category"}


    categorized_tp_sl = {col: categorize_tp_sl(col) for col in STATUS}

    return categorized_tp_sl











TP_SL_OPTIONS, RL_RW_OPTIONS, STD_WINDOW, FIB_RATIOS                = DATA_PULL()
UNIQUE_ITEMS                                                        = [COL for COL in TP_COLS if "TP_LONG_" in COL]
UNIQUE_ITEMS                                                        = [COL.replace("TP_LONG_", "") for COL in UNIQUE_ITEMS]
KEEP_STRATEGIES, KEEP_SCENARIOS                                     = [], []


for PROFIT_LOSS in UNIQUE_ITEMS:
    IMPLEMENTED_PROFIT      = ANALYSIS([f'STATUS__TP_LONG_{PROFIT_LOSS}'], TP_SL_OPTIONS, RL_RW_OPTIONS, STD_WINDOW, FIB_RATIOS)

    if any("TP_Value" in val for val in IMPLEMENTED_PROFIT.values()):
        for first_level_key in IMPLEMENTED_PROFIT: 
            if (IMPLEMENTED_PROFIT[first_level_key]['TP_Value'] <= 1 or IMPLEMENTED_PROFIT[first_level_key]['SL_Value'] <= 1): KEEP_SCENARIOS.append(PROFIT_LOSS)


TAKE_LIST = [x for x in UNIQUE_ITEMS if x not in KEEP_SCENARIOS]


print(len(UNIQUE_ITEMS))
print(len(TAKE_LIST))




104
93


In [76]:

STRATEGIES_GROUPS                           = []
TIMEFRAME_RD                                = ['4H']    #['1D']
COINS                                       = ['AVAXUSDT']
#COINS                                      = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'DOGEUSDT', 'ADAUSDT', ]

DATA_RDC                                    = READ_IN_DATA(COINS, ['4H'], PATH_CLEAN)
DATA_RDS                                    = READ_IN_DATA(COINS, TIMEFRAME_RD, PATH_STRATEGIES)





for X in range(len(COINS)):
    STRATEGY_NBR_                          = [COL for COL in DATA_RDS.LIST[X][0].columns if COL.startswith('STRATEGY_NBR_')] 
    STRATEGIES_LIST_GROUP                  = []

    for STRAT in STRATEGY_NBR_:
            for PROFIT_LOSS in UNIQUE_ITEMS:
                    ELEMENT = [STRAT, PROFIT_LOSS]
                    STRATEGIES_LIST_GROUP.append(ELEMENT)

    STRATEGIES_GROUPS.append(STRATEGIES_LIST_GROUP)

    print(len(STRATEGIES_LIST_GROUP))

157664


In [78]:

PROCEED = False

if PROCEED == True:
        

    BACKTEST_RESULTS_DF                         = pd.DataFrame(columns=["FILE", "STRATEGY", "TP_SL", "END_BALANCE__1", "END_BALANCE__2", "END_BALANCE__3", "WIN_RATE__1", "WIN_RATE__2", "WIN_RATE__3", "NBR_TRADES__1", "NBR_TRADES__2", "NBR_TRADES__3", "MAX_END_BALANCE"])


    print("NBR COINS : " + str(len(COINS)))
    print("NBR TIMEFRAMES : " + str(len(TIMEFRAME_RD)))
    print("----------------------------")
    print("")
    print("")



    for x in range(len(COINS)):
            FILENAME                            = f'{COINS[x]} -- {TIMEFRAME_RD[0]}'
            COUNTER                             = 0


            for A in range(len(STRATEGIES_GROUPS[x])):
                        
                        STRATEGY_USE                = STRATEGIES_GROUPS[x][A][0]
                        TP_SL_USE                   = STRATEGIES_GROUPS[x][A][1]
                        COUNTER                     = COUNTER + 1
                        DATA_DF_TEST                = DATA_RDS.LIST[x][0].copy()
                        DATA_DF_START               = DATA_RDC.LIST[x][0].copy()

                        print(f'{FILENAME}    -   {STRATEGY_USE}    -   {TP_SL_USE}    -   {COUNTER} of {len(STRATEGIES_GROUPS[x])}    -   {round((COUNTER/len(STRATEGIES_GROUPS[x])) * 100, 2)} %')

                        RETAIN_COLS                 = ["DATE", "OPEN", "HIGH", "LOW", "CLOSE", "VOLUME", 'SIGNAL', 'TP', 'SL']
                        CONDENSED_COLS              = [STRATEGY_USE, f'TP_LONG_{TP_SL_USE}', f'SL_LONG_{TP_SL_USE}', f'TP_SHORT_{TP_SL_USE}', f'SL_SHORT_{TP_SL_USE}']
                        DATA_DF_TEST['TP']          = np.where((DATA_DF_TEST[STRATEGY_USE] == "LONG"), DATA_DF_TEST[f'TP_LONG_{TP_SL_USE}'], DATA_DF_TEST[f'TP_SHORT_{TP_SL_USE}'])
                        DATA_DF_TEST['SL']          = np.where((DATA_DF_TEST[STRATEGY_USE] == "LONG"), DATA_DF_TEST[f'SL_LONG_{TP_SL_USE}'], DATA_DF_TEST[f'SL_SHORT_{TP_SL_USE}'])
                        DATA_DF_TEST['SIGNAL']      = DATA_DF_TEST[STRATEGY_USE]
                        DATA_DF_TEST                = DATA_DF_TEST[RETAIN_COLS]
                        DATA_INPUT_LIST             = [DATA_DF_START, DATA_DF_TEST]
                        BACKTEST_OUTPUT             = BACKTEST_NEW(DATA_INPUT_LIST, '2022-01-01', 5, 0.5, 100)
                        APPEND_DICT                 = { "FILE"              : FILENAME,
                                                        "STRATEGY"          : STRATEGY_USE, 
                                                        "TP_SL"             : TP_SL_USE, 
                                                        "END_BALANCE__1"    : round(BACKTEST_OUTPUT.API_DATA.at[0, 'END_BALANCE'],2),
                                                        "END_BALANCE__2"    : round(BACKTEST_OUTPUT.API_DATA.at[1, 'END_BALANCE'],2),
                                                        "END_BALANCE__3"    : round(BACKTEST_OUTPUT.API_DATA.at[2, 'END_BALANCE'],2),
                                                        "WIN_RATE__1"       : round(BACKTEST_OUTPUT.API_DATA.at[0, 'WIN_RATE'],2),
                                                        "WIN_RATE__2"       : round(BACKTEST_OUTPUT.API_DATA.at[1, 'WIN_RATE'],2),
                                                        "WIN_RATE__3"       : round(BACKTEST_OUTPUT.API_DATA.at[2, 'WIN_RATE'],2),
                                                        "NBR_TRADES__1"     : round(BACKTEST_OUTPUT.API_DATA.at[0, 'NBR_TRADES'],2),
                                                        "NBR_TRADES__2"     : round(BACKTEST_OUTPUT.API_DATA.at[1, 'NBR_TRADES'],2),
                                                        "NBR_TRADES__3"     : round(BACKTEST_OUTPUT.API_DATA.at[2, 'NBR_TRADES'],2),
                                                        "MAX_END_BALANCE"   : round(BACKTEST_OUTPUT.API_DATA['END_BALANCE'].max(),2)}

                        BACKTEST_RESULTS_DF.loc[len(BACKTEST_RESULTS_DF)] = APPEND_DICT
                        if COUNTER % 1000 == 0: DF_OUT = EXPORT_RAW_DATA(BACKTEST_RESULTS_DF, (PATH_OUTPUT + f'BACKTEST_RESULTS_{TIMEFRAME_RD[0]}.csv'))


    DF_OUT = EXPORT_RAW_DATA(BACKTEST_RESULTS_DF, (PATH_OUTPUT + f'BACKTEST_RESULTS_{TIMEFRAME_RD[0]}.csv'))



In [61]:
DF_OUT = EXPORT_RAW_DATA(BACKTEST_RESULTS_DF, (PATH_OUTPUT + f'BACKTEST_RESULTS_{TIMEFRAME_RD[0]}.csv'))

Data has been exported to - /Users/westhomas/Desktop/ENKI/ENKI_CODE_DEVELOPMENT/D__RAW_DATA_OUTPUT/BACKTEST_RESULTS_1D.csv


In [52]:
import pandas as pd




#BACKTEST_RESULTS_DF = BACKTEST_RESULTS_DF[BACKTEST_RESULTS_DF['STRATEGY']!='STRATEGY_NBR_246']
#BACKTEST_RESULTS_DF = BACKTEST_RESULTS_DF[BACKTEST_RESULTS_DF['END_BALANCE__1']>1000]
#BACKTEST_RESULTS_DF = BACKTEST_RESULTS_DF[BACKTEST_RESULTS_DF['END_BALANCE__2']>1000]





TIMEFRAME_RESULTS   = "4H"
BACKTEST_RESULTS_DF = pd.read_csv(PATH_OUTPUT + f'BACKTEST_RESULTS_{TIMEFRAME_RESULTS}.csv') 
BACKTEST_RESULTS_DF = BACKTEST_RESULTS_DF.sort_values(by=["MAX_END_BALANCE", "END_BALANCE__2", "END_BALANCE__1"], ascending=[False, False, False])


BACKTEST_RESULTS_DF[:50]



,FILE,STRATEGY,TP_SL,END_BALANCE__1,END_BALANCE__2,END_BALANCE__3,WIN_RATE__1,WIN_RATE__2,WIN_RATE__3,NBR_TRADES__1,NBR_TRADES__2,NBR_TRADES__3,MAX_END_BALANCE
0,BTCUSDT -- 4H,STRATEGY_NBR_1,1,69.25,10.52,69.25,0.48,0.29,0.48,122,122,122,69.25
1,BTCUSDT -- 4H,STRATEGY_NBR_1,2,80.32,10.52,77.60,0.61,0.29,0.60,120,122,122,80.32
2,BTCUSDT -- 4H,STRATEGY_NBR_1,3,108.20,10.52,106.57,0.71,0.29,0.70,120,122,122,108.20
3,BTCUSDT -- 4H,STRATEGY_NBR_1,4,121.46,10.52,131.83,0.76,0.29,0.75,118,122,122,131.83
4,BTCUSDT -- 4H,STRATEGY_NBR_1,5,117.13,10.52,125.55,0.87,0.29,0.80,104,122,122,125.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,BTCUSDT -- 4H,STRATEGY_NBR_2,9,165.49,85.32,178.72,0.67,0.34,0.66,85,90,90,178.72
113,BTCUSDT -- 4H,STRATEGY_NBR_2,10,224.34,85.32,235.87,0.75,0.34,0.72,84,90,90,235.87
114,BTCUSDT -- 4H,STRATEGY_NBR_2,11,122.77,85.32,218.66,0.81,0.34,0.78,79,90,90,218.66
115,BTCUSDT -- 4H,STRATEGY_NBR_2,12,34.76,85.32,147.28,0.85,0.34,0.78,72,90,90,147.28
